In [22]:
! pip install cohere langchain qdrant-client tfds-nightly python-dotenv pymongo pandas numpy

In [1]:

from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from langchain import PromptTemplate, LLMChain
import textwrap as tr
import random
import dotenv
import os
import pymongo
import pandas as pd

dotenv.load_dotenv(".env")

COHERE_API_KEY = os.getenv("COHERE_API_KEY")
MONGO_URI = os.getenv("MONGO_URI")

### Load data into MongoDB

In [2]:
df = pd.read_csv("data/okcupid_profiles.csv")
df.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,NaN,...,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,white,...,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",...,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN


In [5]:
# Save dataframe in MongoDB
mongodb_client = pymongo.MongoClient(MONGO_URI, serverSelectionTimeoutMS=3_600_000)

# Create dataset and collection
db = mongodb_client.cupidgpt
collection = db.profiles

# Insert dataframe into collection
collection.insert_many(df.to_dict("records"))

# Delete all documents in collection
# collection.delete_many({})




AutoReconnect: ac-1rncjhl-shard-00-01.cej0daj.mongodb.net:27017: connection closed

### Cohere GenAI LLM

In [13]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = Cohere(cohere_api_key=COHERE_API_KEY)
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(question)

' Justin Bieber was born on March 1, 1994.\n\nThe Super Bowl is an annual championship game of the NFL that is played in late January or early February.\n\nTherefore, we are looking for the NFL team that won the Super Bowl in the year 1994.\n\nIn 1994, Super Bowl XXVIII was played on January 30, 1994, between the Dallas Cowboys and the Buffalo Bills.\n\nThe Dallas Cowboys won Super Bowl XXVIII, defeating the Buffalo Bills 30-13.\n\nSo, the answer is the Dallas Cowboys.'